In [1]:
from chemwalker.gnps import Proteosafe
from chemwalker.utils import run_metfrag, walk_conn_comp
import pandas as pd
import numpy as np
from rdkit import Chem

In [2]:
taskid = 'be5004d4dd394fe8b883633fd75ea732'

gnps_result = Proteosafe(taskid, 'V2')
gnps_result.get_gnps()
net = gnps_result.net
tabgnps = gnps_result.gnps
spectra = gnps_result.spectra
dbmatch = gnps_result.dbmatch

In [3]:
net.head()

,CLUSTERID1,CLUSTERID2,DeltaMZ,MEH,Cosine,OtherScore,ComponentIndex,EdgeAnnotation
0,549,574,0.000,549.0,0.9488,549.0,1,
1,10,574,-15.995,10.0,0.8224,10.0,1,
2,10,549,-15.995,10.0,0.7616,10.0,1,
3,2,390,-84.147,2.0,0.8661,2.0,3,
4,1,2,-42.967,1.0,0.9744,1.0,3,


In [4]:
tabgnps.head()

,AllGroups,DefaultGroups,EvenOdd,G1,G2,G3,G4,G5,G6,GNPSLinkout_Cluster,...,SpectrumID,UniqueFileSources,UniqueFileSourcesCount,cluster index,componentindex,number of spectra,parent mass,precursor charge,precursor mass,sum(precursor intensity)
0,,G1,1,1,0,0,0,0,0,https://gnps.ucsd.edu//ProteoSAFe/result.jsp?t...,...,CCMSLIB00003153831,gnps_net_filtered_net1.mgf,1,1,3,1,319.086,1,319.086,0
1,,G1,0,1,0,0,0,0,0,https://gnps.ucsd.edu//ProteoSAFe/result.jsp?t...,...,CCMSLIB00003153832,gnps_net_filtered_net1.mgf,1,2,3,1,362.053,1,362.053,0
2,,G1,1,1,0,0,0,0,0,https://gnps.ucsd.edu//ProteoSAFe/result.jsp?t...,...,CCMSLIB00003153833,gnps_net_filtered_net1.mgf,1,3,3,1,311.027,1,311.027,0
3,,G1,1,1,0,0,0,0,0,https://gnps.ucsd.edu//ProteoSAFe/result.jsp?t...,...,NaN,gnps_net_filtered_net1.mgf,1,4,55,1,393.135,1,393.135,0
4,,G1,1,1,0,0,0,0,0,https://gnps.ucsd.edu//ProteoSAFe/result.jsp?t...,...,CCMSLIB00003153946,gnps_net_filtered_net1.mgf,1,5,94,1,310.216,1,310.216,0


In [5]:
tabgnps.shape

(610, 28)

In [6]:
len(spectra)

610

In [7]:
spectra[0]

{'params': {'pepmass': (319.08591, None),
  'charge': [1],
  'mslevel': '2',
  'filename': 'spec-00000.mgf',
  'instrument': 'ion trap',
  'title': 'Scan Number: 1',
  'scans': '1'},
 'm/z array': array([ 94., 141., 153., 180., 181., 185.]),
 'intensity array': array([  2.,   3., 111.,   3., 999.,   1.]),
 'charge array': masked_array(data=[--, --, --, --, --, --],
              mask=[ True,  True,  True,  True,  True,  True],
        fill_value=0,
             dtype=int64)}

In [8]:
db = pd.read_csv('validation_db.psv', sep='|')
nist = pd.read_csv('nist17_selected_db_ccms.txt', sep='\t')
db = pd.concat([db, nist]).reset_index(drop=True)
db.head()

,MonoisotopicMass,InChI,SMILES,Identifier,InChIKey2,InChIKey1,MolecularFormula,kingdom_name,superclass_name,class_name,subclass_name
0,194.05791,InChI=1/C10H10O4/c1-14-9-6-7(2-4-8(9)11)3-5-10...,COC1=C(C=CC(=C1)C=CC(=O)O)O,CCMSLIB00000220714,UHFFFAOYSA,KSEBMYQBYZTDHS,C10H10O4,Organic compounds,Phenylpropanoids and polyketides,Cinnamic acids and derivatives,Hydroxycinnamic acids and derivatives
1,176.09496,InChI=1/C10H12N2O/c11-4-3-7-6-12-10-2-1-8(13)5...,C1=CC2=C(C=C1O)C(=CN2)CCN,CCMSLIB00000220735,UHFFFAOYSA,QZAYGJVTTNCVMB,C10H12N2O,Organic compounds,Organoheterocyclic compounds,Indoles and derivatives,Tryptamines and derivatives
2,176.04734,InChI=1/C10H8O3/c1-6-4-10(12)13-9-5-7(11)2-3-8...,CC1=CC(=O)OC2=C1C=CC(=C2)O,CCMSLIB00000221773,UHFFFAOYSA,HSHNITRMYYLLCV,C10H8O3,NaN,NaN,NaN,NaN
3,345.07864,"InChI=1/C12H17N4O4PS/c1-8-11(3-4-20-21(17,18)1...",CC1=C(SC=[N+]1CC2=CN=C(N=C2N)C)CCOP(=O)(O)O,CCMSLIB00000221098,UHFFFAOYSA,HZSAJDVWZRBGIF,C12H18N4O4PS,Organic compounds,Organoheterocyclic compounds,Diazines,Pyrimidines and pyrimidine derivatives
4,342.11621,InChI=1/C12H22O11/c13-1-4-6(15)8(17)9(18)11(22...,C(C1C(C(C(C(O1)OCC2C(C(C(O2)(CO)O)O)O)O)O)O)O,CCMSLIB00000222747,UHFFFAOYSA,PVXPPJIGRGXGCY,C12H22O11,NaN,NaN,NaN,NaN


In [9]:
nset = list(set(net['ComponentIndex']))
nset[4]

6

In [10]:
net['ComponentIndex'].value_counts()

6     321
4     173
26     93
33     68
23     60
     ... 
35      1
71      1
72      1
61      1
98      1
Name: ComponentIndex, Length: 96, dtype: int64

In [11]:
dbmatch.head()

,#Scan#,Adduct,CAS_Number,Charge,Compound_Name,Compound_Source,Data_Collector,ExactMass,FileScanUniqueID,INCHI,...,RT_Query,SharedPeaks,Smiles,SpecCharge,SpecMZ,SpectrumFile,SpectrumID,TIC_Query,UpdateWorkflowName,tags
0,1,M+H,NaN,1,"2-((4,6-Dimethyl-2-pyrimidinyl)thio)-N-(3-nitr...",Standard,NIST,318.079,spectra/specs_ms.pklbin1,InChI=1S/C14H14N4O3S/c1-9-6-10(2)16-14(15-9)22...,...,0,6,CC1=CC(=NC(=N1)SCC(=O)NC2=CC(=CC=C2)[N+](=O)[O...,1,319.086,spectra/specs_ms.pklbin,CCMSLIB00003153831,1119.00,UPDATE-SINGLE-ANNOTATED-GOLD,
1,10,M+H,NaN,1,Promethazine,Standard,NIST,284.135,spectra/specs_ms.pklbin10,InChI=1S/C17H20N2S/c1-13(18(2)3)12-19-14-8-4-6...,...,0,15,CC(CN1C2=CC=CC=C2SC3=CC=CC=C31)N(C)C,1,285.142,spectra/specs_ms.pklbin,CCMSLIB00003161305,2553.64,UPDATE-SINGLE-ANNOTATED-GOLD,
2,100,M+H,NaN,1,N-(15Z-Tetracosenoyl)-1-.beta.-galactosylsphin...,Standard,NIST,809.674,spectra/specs_ms.pklbin100,InChI=1S/C48H91NO8/c1-3-5-7-9-11-13-15-17-18-1...,...,0,22,CCCCCCCCCCCCCC=CC(C(COC1C(C(C(C(O1)CO)O)O)O)NC...,1,810.682,spectra/specs_ms.pklbin,CCMSLIB00003154790,1463.06,UPDATE-SINGLE-ANNOTATED-GOLD,
3,101,M+H,NaN,1,.alpha.-Galactosyl-N-stearoylsphingosine,Standard,NIST,727.596,spectra/specs_ms.pklbin101,InChI=1S/C42H81NO8/c1-3-5-7-9-11-13-15-17-18-2...,...,0,21,CCCCCCCCCCCCCCCCCC(=O)NC(COC1C(C(C(C(O1)CO)O)O...,1,728.604,spectra/specs_ms.pklbin,CCMSLIB00003154791,2520.19,UPDATE-SINGLE-ANNOTATED-GOLD,
4,102,M+H,NaN,1,Gln-Gly,Standard,NIST,203.091,spectra/specs_ms.pklbin102,InChI=1S/C7H13N3O4/c8-4(1-2-5(9)11)7(14)10-3-6...,...,0,11,C(CC(=O)N)C(C(=O)NCC(=O)[O-])[NH3+],1,204.098,spectra/specs_ms.pklbin,CCMSLIB00003154823,1771.23,UPDATE-SINGLE-ANNOTATED-GOLD,


In [12]:
dbmatch.fillna('', inplace=True)

# Define fixed number o seeds to start walk, mostly for validation
seed_ctr = 0.1 # 10% of seeds in a given connected component
seed_ctr = np.ceil(sum(dbmatch['INCHI']!='')*seed_ctr)
seed_ctr = int(seed_ctr)

# Remove known structures for validation
exid =  dbmatch.index[seed_ctr:]
dbmatch.loc[exid, 'INCHI'] = ''
dbmatch.loc[exid, 'Smiles'] = ''
dbmatch.tail()

,#Scan#,Adduct,CAS_Number,Charge,Compound_Name,Compound_Source,Data_Collector,ExactMass,FileScanUniqueID,INCHI,...,RT_Query,SharedPeaks,Smiles,SpecCharge,SpecMZ,SpectrumFile,SpectrumID,TIC_Query,UpdateWorkflowName,tags
586,94,M+H,,1,3-Hydroxy-4-methoxycinnamicacid,Standard,NIST,194.058,spectra/specs_ms.pklbin94,,...,0,10,,1,195.065,spectra/specs_ms.pklbin,CCMSLIB00003154674,2210.31,UPDATE-SINGLE-ANNOTATED-GOLD,
587,95,M+H,,1,4-Cholestenone,Standard,NIST,384.339,spectra/specs_ms.pklbin95,,...,0,17,,1,367.300,spectra/specs_ms.pklbin,CCMSLIB00003161068,6472.41,UPDATE-SINGLE-ANNOTATED-GOLD,
588,96,M+H,,1,Adipicacid,Standard,NIST,146.058,spectra/specs_ms.pklbin96,,...,0,8,,1,147.065,spectra/specs_ms.pklbin,CCMSLIB00003154675,2923.16,UPDATE-SINGLE-ANNOTATED-GOLD,
589,97,M+H,,1,Acetyl-L-carnitine,Standard,NIST,203.116,spectra/specs_ms.pklbin97,,...,0,10,,1,204.123,spectra/specs_ms.pklbin,CCMSLIB00003154694,1128.29,UPDATE-SINGLE-ANNOTATED-GOLD,
590,98,M+H,,1,Maltotetraose,Standard,NIST,666.222,spectra/specs_ms.pklbin98,,...,0,27,,1,667.229,spectra/specs_ms.pklbin,CCMSLIB00003154735,1360.71,UPDATE-SINGLE-ANNOTATED-GOLD,


In [ ]:
tlid = walk_conn_comp(net=net, spectra=spectra, tabgnps=tabgnps, 
                      dbmatch=dbmatch, db=db, comp_index=6)

Component with 87 nodes
Calculating in silico fragmentation with MetFrag...
in silico fragmentation done in: 540.4001240730286 seconds
Calculating pairwise candidate similarities...
